In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.initializers import glorot_normal, HeNormal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from tensorflow.keras.utils import set_random_seed

# For reproducible results set random seed
set_random_seed(43)
tf.keras.backend.set_floatx('float32')

# Training

In [ ]:
# Create Custom Loss Functions
def custom_mean_absolute_error(y_true, y_pred):
    return tf.reduce_mean(tf.norm(y_true-y_pred, axis=-1))

def custom_median_absolute_error(y_true, y_pred):
    return tfp.stats.percentile(tf.norm(y_true-y_pred, axis=-1), 50.0, interpolation='midpoint')

def custom_mean_squared_error(y_true, y_pred):
  return tf.reduce_mean(tf.norm(y_true-y_pred, axis=-1)**2)

def custom_median_squared_error(y_true, y_pred):
  return tfp.stats.percentile(tf.norm(y_true-y_pred, axis=-1)**2, 50.0, interpolation='midpoint')

In [ ]:
# Define the model
model = Sequential()

# Input layer
model.add(Dense(12, input_dim=2, activation='relu'))

# Hidden layers
model.add(Dense(24, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))

# Output layer with 2 neurons for x and y coordinates
model.add(Dense(2))

model.compile(optimizer = Adam(learning_rate=10**-4),
              loss = custom_mean_absolute_error,
              metrics = ['accuracy'])

# Import train/val data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/The chosen ones/rand_data_v4.csv')

# Separate the input features (X) and output labels (y)
X = data.iloc[:10**6, 4:].values
Y = data.iloc[:10**6, :2].values

# Split the data into training and testing sets
# Constant random state is needed in order to have the same split each time
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15)

# Prepare the data for the NN, using Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Train the model
history = model.fit(X_train_scaled, Y_train, epochs=50, batch_size=64, validation_data=(X_val_scaled, Y_val), shuffle=True)

In [ ]:
# Visualize the results
import matplotlib.pyplot as plt
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create x-axis values (epochs)
epochs = range(1, len(train_loss) + 1)

# Plot training & validation loss
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (m)')
plt.ylim([0, 20])

plt.legend()

# Display the plot
plt.show()


In [ ]:
# Save the entire model to a HDF5 file.C:\Users\antik\My Drive\Colab Notebooks\The chosen ones\Working Projects\No Amb
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('/content/drive/MyDrive/Colab Notebooks/The chosen ones/Working Projects/No Amb/Non-Optimal/relu_Dphi_mae_43.keras')

# Save Train/Val. losses
train_val_loss = pd.DataFrame(
    {'train_loss': train_loss,
     'validation_loss': val_loss
    })
train_val_loss.to_csv('/content/drive/MyDrive/Colab Notebooks/The chosen ones/Working Projects/No Amb/Non-Optimal/relu_Dphi_mae_losses43.csv')

# Testing

In [ ]:
# Rand. Tag Position
# For .csv files with more than a million samples
# Separate the input features (X) and output labels (y)
X_test = data.iloc[10**6+1:, 4:].values
Y_test = data.iloc[10**6+1:, :2].values
X_test_scaled = scaler.transform(X_test)

loss_rand = model.evaluate(X_test_scaled, Y_test)
print(f'MAE  Test loss: {loss_rand}')
print(f'RMSE Test loss: {tf.sqrt(loss_rand)}')

predictions_rand = model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions_rand)

# save the dataframe as a csv file
predictions_df.to_csv("/content/drive/MyDrive/Colab Notebooks/The chosen ones/Predictions/No Amb/Non-Optimal/relu_Dphi_mae_predictions_rand_43.csv")

# Calculate all errors
error = custom_mean_absolute_error(Y_test, predictions_rand)
print(f'Mean Absolute Error: {error}')

error = custom_median_absolute_error(Y_test, predictions_rand)
print(f'Median Absolute loss: {error}')

error = custom_mean_squared_error(Y_test, predictions_rand)
print(f'Root Mean Squared Error: {tf.sqrt(error)}')

error = custom_median_squared_error(Y_test, predictions_rand)
print(f'Root Median Squared Error: {tf.sqrt(error)}')

In [ ]:
# Scenario 1
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/The chosen ones/grid_data_scenario1_v4.csv')

# Separate the input features (X) and output labels (y)
X_test = test_data.iloc[:, 4:].values
Y_test = test_data.iloc[:, :2].values
X_test_scaled = scaler.transform(X_test)

loss_scenario1 = model.evaluate(X_test_scaled, Y_test)
print(f'MAE  Test loss: {loss_scenario1}')
print(f'RMSE Test loss: {tf.sqrt(loss_scenario1)}')

predictions_scenario1 = model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions_scenario1)

# save the dataframe as a csv file
predictions_df.to_csv("/content/drive/MyDrive/Colab Notebooks/The chosen ones/Predictions/No Amb/Non-Optimal/relu_Dphi_mae_predictions_scenario1_43.csv")

# Calculate all errors
error = custom_mean_absolute_error(Y_test, predictions_scenario1)
print(f'Mean Absolute Error: {error}')

error = custom_median_absolute_error(Y_test, predictions_scenario1)
print(f'Median Absolute loss: {error}')

error = custom_mean_squared_error(Y_test, predictions_scenario1)
print(f'Root Mean Squared Error: {tf.sqrt(error)}')

error = custom_median_squared_error(Y_test, predictions_scenario1)
print(f'Root Median Squared Error: {tf.sqrt(error)}')

In [ ]:
# Scenario 2
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/The chosen ones/grid_data_scenario2_v4.csv')

# Separate the input features (X) and output labels (y)
X_test = test_data.iloc[:, 4:].values
Y_test = test_data.iloc[:, :2].values
X_test_scaled = scaler.transform(X_test)

loss_scenario2 = model.evaluate(X_test_scaled, Y_test)
print(f'MAE  Test loss: {loss_scenario2}')
print(f'RMSE Test loss: {tf.sqrt(loss_scenario2)}')

predictions_scenario2 = model.predict(X_test_scaled)
predictions_df = pd.DataFrame(predictions_scenario2)

# save the dataframe as a csv file
predictions_df.to_csv("/content/drive/MyDrive/Colab Notebooks/The chosen ones/Predictions/No Amb/Non-Optimal/relu_Dphi_mae_predictions_scenario2_43.csv")

# Calculate all errors
error = custom_mean_absolute_error(Y_test, predictions_scenario2)
print(f'Mean Absolute Error: {error}')

error = custom_median_absolute_error(Y_test, predictions_scenario2)
print(f'Median Absolute loss: {error}')

error = custom_mean_squared_error(Y_test, predictions_scenario2)
print(f'Root Mean Squared Error: {tf.sqrt(error)}')

error = custom_median_squared_error(Y_test, predictions_scenario2)
print(f'Root Median Squared Error: {tf.sqrt(error)}')